# Prototype validation tools
In developing a new data validation framework, it's important to demonstrate that existing validations can be translated into the new framework. This notebook will demonstrate a factory pattern for generating `asset_check`'s which will run [Great Expectations](https://docs.greatexpectations.io/docs/core/introduction/) under the hood for easy configurable validations.

In [ ]:
!pip install great_expectations

## Helper tools
Define some useful tools for prepping tests.

In [ ]:
from pudl.etl import defs
import pandas as pd

def _get_asset(asset_key: str) -> pd.DataFrame:
    return defs.load_asset_value(asset_key)

## Define `asset_check` factory
Define a fake `asset_check` factory to generate GX based tests. This won't create real `asset_check`'s because they can't easily be run in a notebook, but is meant to mirror what the API would look like.

In [ ]:
import great_expectations as gx
from typing import Type, Any
from dataclasses import dataclass

@dataclass
class ValidationResult:
    """This would be an AssetCheckResult in production."""
    passed: bool
    metadata: dict
    description: str | None
    context: Any

def validation_factory(
    asset_name: str,
    expectation: Type[gx.expectations.Expectation],
    expectation_config: dict,
    fast_etl_expectation_config: dict | None = None,
    description: str | None = None,
):
    """Return a function which will execute a great expectations expectation."""
    def _validation():
        df = _get_asset(asset_name)

        # Connect to data
        context = gx.get_context()
        batch = context.data_sources.pandas_default.read_dataframe(df)

        # Create expectation (in actual asset_check factory this would check job name to supply correct config)
        configured_expectation = expectation(**expectation_config)

        # Run test
        validation_result = batch.validate(configured_expectation)

        return ValidationResult(
            passed=validation_result.success,
            description=description,
            metadata=validation_result.result,
            context=context,
        )

In [ ]:
validation = validation_factory(
    "out_ferc1__yearly_steam_plants_fuel_by_plant_sched402",
    gx.exp